In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from utils import delete_patients_with_the_same_GUID, averaging_scores
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

Observe some patients have different UPDRS score parogression types, and a natural question will be whether some medication will have different effect on them or not. We prepara the data to do a visualization for that. (Need revision?)

In [21]:
enrollment = delete_patients_with_the_same_GUID(pd.read_csv("new-data/releases_2023_v4release_1027_clinical_Enrollment.csv"))

updrs_scores = pd.read_csv("cleaned_data.csv")[['participant_id', 'visit_name', 'visit_month',
       'mds_updrs_part_i_summary_score', 'mds_updrs_part_ii_summary_score',
       'mds_updrs_part_iii_summary_score']]

enrollment_pd = enrollment[enrollment['study_arm'] == 'PD']

updrs_scores = updrs_scores[updrs_scores['participant_id'].isin(enrollment_pd['participant_id'])]

updrs_scores.columns = ['participant_id', 'visit_name', 'visit_month', 'updrs_1', 'updrs_2', 'updrs_3']

updrs_scores = averaging_scores(updrs_scores, ['updrs_1', 'updrs_2', 'updrs_3'])

updrs_scores = updrs_scores.drop(columns = ['visit_name'])

updrs_scores.to_csv("cleaned_updrs_scores.csv", index=False)

Processing screening records by participant...
Consolidated 474 screening records into 457 baseline records.
Processing regular visit records by participant and month...
Consolidated 10864 regular records into 10633 records.


In [22]:
med_data = delete_patients_with_the_same_GUID(pd.read_csv("new-data/releases_2023_v4release_1027_clinical_PD_Medical_History.csv"))

med_data = med_data[med_data['participant_id'].isin(enrollment_pd['participant_id'])]

med_columns = ['on_levodopa', 'on_dopamine_agonist', 'on_other_pd_medications']
for col in med_columns:
    med_data[col] = med_data[col].apply(lambda x: 1 if x == 'Yes' else 0)

med_data = med_data.groupby('participant_id').agg({
    'on_levodopa': 'max',
    'on_dopamine_agonist': 'max',
    'on_other_pd_medications': 'max'
}).reset_index()

med_data.to_csv("cleaned_med_data.csv", index=False)

In [23]:
med_data.on_levodopa.value_counts()

on_levodopa
1    1730
0     966
Name: count, dtype: int64

In [36]:
MOCA_data = delete_patients_with_the_same_GUID(pd.read_csv("new-data/releases_2023_v4release_1027_clinical_MOCA.csv"))

MOCA_data = MOCA_data[MOCA_data['participant_id'].isin(enrollment_pd['participant_id'])]





In [37]:
MOCA_data.columns

Index(['participant_id', 'GUID', 'visit_name', 'visit_month',
       'moca01_alternating_trail_making', 'moca02_visuoconstr_skills_cube',
       'moca03_visuoconstr_skills_clock_cont',
       'moca04_visuoconstr_skills_clock_num',
       'moca05_visuoconstr_skills_clock_hands',
       'moca_visuospatial_executive_subscore', 'moca06_naming_lion',
       'moca07_naming_rhino', 'moca08_naming_camel', 'moca_naming_subscore',
       'moca09_attention_forward_digit_span',
       'moca10_attention_backward_digit_span',
       'moca_attention_digits_subscore', 'moca11_attention_vigilance',
       'moca12_attention_serial_7s', 'moca13_sentence_repetition',
       'moca14_verbal_fluency_number_of_words', 'moca15_verbal_fluency',
       'moca_language_subscore', 'moca16_abstraction',
       'moca_abstraction_subscore', 'moca17_delayed_recall_face',
       'moca18_delayed_recall_velvet', 'moca19_delayed_recall_church',
       'moca20_delayed_recall_daisy', 'moca21_delayed_recall_red',
       'moca

In [38]:
MOCA_data['visit_month'].unique()

array([  0,  12,  24,  36,  48,  60,  84,  96,  -1,  72, 108,  30,  18,
         9,   3])

In [43]:
MOCA_data.columns

Index(['participant_id', 'GUID', 'visit_name', 'visit_month',
       'moca01_alternating_trail_making', 'moca02_visuoconstr_skills_cube',
       'moca03_visuoconstr_skills_clock_cont',
       'moca04_visuoconstr_skills_clock_num',
       'moca05_visuoconstr_skills_clock_hands',
       'moca_visuospatial_executive_subscore', 'moca06_naming_lion',
       'moca07_naming_rhino', 'moca08_naming_camel', 'moca_naming_subscore',
       'moca09_attention_forward_digit_span',
       'moca10_attention_backward_digit_span',
       'moca_attention_digits_subscore', 'moca11_attention_vigilance',
       'moca12_attention_serial_7s', 'moca13_sentence_repetition',
       'moca14_verbal_fluency_number_of_words', 'moca15_verbal_fluency',
       'moca_language_subscore', 'moca16_abstraction',
       'moca_abstraction_subscore', 'moca17_delayed_recall_face',
       'moca18_delayed_recall_velvet', 'moca19_delayed_recall_church',
       'moca20_delayed_recall_daisy', 'moca21_delayed_recall_red',
       'moca

In [42]:
MOCA_data['visit_name'].unique()

array(['M0', 'M12', 'M24', 'M36', 'M48', 'M60', 'M84', 'M96', 'SC', 'M72',
       'M108', 'M30', 'M18', 'M9', 'M3'], dtype=object)

In [45]:
MOCA_data_scores = MOCA_data[MOCA_data['visit_month'] >= 0][['participant_id','visit_month','moca_total_score']]

In [46]:
MOCA_data_scores.head()

,participant_id,visit_month,moca_total_score
1,BF-1002,0,29.0
2,BF-1003,0,30.0
3,BF-1004,0,28.0
5,BF-1006,0,25.0
6,BF-1008,0,29.0


In [48]:
MOCA_data_scores.to_csv("MOCA_cleaned.csv", index = False)